In [1]:
#Batch gradient descent optimisation algorithm

from sympy import *
m, c = symbols('m c')
exp = '0.5*(y-m*x-c)**2'
exp_diff = Derivative(exp,m)
dfm = exp_diff.doit()
exp_diff = Derivative(exp,c)
dfc = exp_diff.doit()
print(f'1st order derivative w.r.t m and c :\nde/dm = {dfm} \nde/dc = {dfc}')
xar = [0.2,0.4,0.6,0.8]
yar = [2.4,3.8,4.2,4.6]

#step 1:
m1=1
c1=-1
learning_rate=0.1
itr=100

for i in range(0,itr):
  dedm1 = 0
  dedc1 = 0
  for j in range(0,len(xar)):
    #step 2:
    m,c,x,y = symbols('m c x y')
    dedm = dfm.subs(m,m1)
    dedm = dedm.subs(c, c1) 
    dedm= dedm.subs(x, xar[j]) 
    dedm = dedm.subs(y, yar[j])
    dedm = round(dedm,2)
    dedc = dfc.subs(c,c1)
    dedc = dedc.subs(m, m1) 
    dedc = dedc.subs(x, xar[j]) 
    dedc = dedc.subs(y, yar[j])
    dedc = round(dedc,2)
    dedm1 += dedm
    dedc1 += dedc

  dedm1 = dedm1/len(xar)
  dedc1 = dedc1/len(xar)
  dedm1 = round(dedm1,2)
  dedc1 = round(dedc1,2)
  #step 3:
  dm = (-0.1)*learning_rate*dedm1
  dc = (-0.1)*learning_rate*dedc1
  #step 4:
  m1 = m1+dm
  m1 = round(m1,2)
  c1 = c1+dc
  c1 = round(c1,2)

print(f'min value of m = {m1} \nmin value of c = {c1}')

1st order derivative w.r.t m and c :
de/dm = -1.0*x*(-c - m*x + y) 
de/dc = 1.0*c + 1.0*m*x - 1.0*y
min value of m = 2.32 
min value of c = 1.39
